---
title: Regression
format:
  html:
      embed-resources: true
      code-fold: true
      cite-method: biblatex
---

## Introduction to Decision Trees

Decision Trees are a supervised learning technique that uses non-parametric methods for classification and regression. This differs from many disciplines of mathematical modeling, as non-parametric statistics do not make assumptions about the distribution of the sample's population. Thus, regardless of the distribution, Decision Trees attempt to create models from the data simply by creating sequential sets of rules. These rules can be interpreted as sets of `if-else` statements, as we would understand them in programming. In this regard, Decision Trees follow the process of creating a conditions, assessing whether or not an observation follows that condition, and assigns an outcome based on the result. This can then be repeated iteratively to the resulting value when applying a different condition, thus creating a set of splits with different trajectories to resemble a tree diagram.

As mentioned above, Decision Trees can be categorized into two methods: classification and regression. Classification Decision Trees will be recapped on the `Classification` tab of this section.

### Regression

Regression trees are specifically used for when a target variable takes on continuous values. To do this, they attempt to estimate the local nature of the underlying function (i.e., how  a particular variable impacts the target at a certain value or range of values), rather than the global shape of the data.

Therefore, each condition will be a question of whether a record fits a value or a range of values for a particular variable, and each node is a predicted numerical output that will estimate the target based on a condition. Due to the discrete nature of decision trees and the fact that the target variable is continuous, this can result in large error, so applying error metrics is crucial.

One method that will be applied below is *pruning*, which is a very common way of simplifying trees. Doing this cuts down on the number of leaves by replacing excess nodes and their respective subtrees with singular leaves. This can significantly reduce the size of a tree, as well as improve performance in cases where accuracy of a training set deteriorates as overfitting begins to occur.

Secondly, we will also be using random forests for improved supervised learning. This is a widely used bagging algorithm, which takes a decision tree and attempts to improve its stability and accuracy. This will mainly be useful for reducing variance and preventing overfitting, and will also serve as a way to evaluate the predictiveness of each input variable.

## Regresssion Trees for Record Data

### Target Distribution

The record data to which we will apply regression Decision Trees to is the dataset of public transit data by city. In this dataset there are two target variables we will be looking at, which will each serve a significant purpose in determining how the public transit system in a city succeeds:

- `Trips_per_capita`: All public transportation trips (bus + train + ferry + ...) divided by total metropolitan area population. Put simply, this is the relative usage of a public transit system. The distribution of this variable is visualized below:

![Trips_per_capita Boxplot](../../images/box1.png)

- `Cost_per_trip`: The monetary value (in USD) that a city spends on its public transit system divided by the total number of trips. This value represents the cost effectiveness of the system, thus providing insights on its efficacy. The distribution of this variable is visualized below:

![Cost_per_trip Boxplot](../../images/box2.png)

The input variables for both of these targets are the following:

- `Population`: The total metropolitan area population of the city
- `Area`: The total area in square miles of the city
- `Fare_per_trip`: How much the customers spend in USD on average on each trip
- `Miles_per_trip`: The average length in miles of each trip

### Applying the Model and Pruning

The code for creating the initial regression tree is below, starting with `Trips_per_capita` as our target variable:

In [ ]:
apta <- read.csv("../../data/cleaned_data/apta_cities_cleaned.csv")
library(tree)
library(rpart)
library(rattle)
library(visNetwork)
set.seed(1)
train = sample(1:nrow(apta), nrow(apta)/2)
apta.tree=tree(Trips_per_capita ~ Population + Area + Fare_per_trip + Miles_per_trip, apta, subset=train)
summary(apta.tree)
apta.tree2=rpart(Trips_per_capita ~ Population + Area + Fare_per_trip + Miles_per_trip, apta ,subset=train)
fancyRpartPlot(apta.tree2)

![Initial Regression Tree](../../images/regression1.png)

There are a few things notable about this initial tree. Firstly, there are very few nodes which indicates a low chance of overfitting, but a strong possibility of high variance. Secondly, we can take a look at which variables are prevalent in the conditions. It is notable that `Fare_per_trip` is not a determining factor in predicting the usage of a public transit system, while variables that are more inherent to a city's characteristics are present. We can learn more via pruning and obtaining another tree:

In [ ]:
cv.apta <- cv.tree(apta.tree)
plot(cv.apta$size,cv.apta$dev,type='b')
prune.apta=prune.tree(apta.tree,best=5)
plot(prune.apta)
text(prune.apta,pretty=0)
yhat=predict(apta.tree,newdata=apta[-train,])
apta.test=apta[-train,"Trips_per_capita"]
plot(yhat,apta.test)
abline(0,1)

![Pruning via cv](../../images/regression1_prune.png)

![Pruned Tree](../../images/regression1_pruned.png)

This pruned tree tells a similar story to the original one. The tree is of similar size, and the same input variables are present.

![Results of Pruned Tree](../../images/regression1_results.png)

From these results, we can see that the tree was only partially effective in predicting `Trips_per_capita`. Especially where the predictions got large, there is quite a bit of error to account for.

Next, we will turn our attention to the inital steps of regression trees with `Cost_per_trip` as the target variable:

In [ ]:
train2 <- sample(1:nrow(apta), nrow(apta)/2)
apta.tree3=tree(Cost_per_trip ~ Population + Area + Fare_per_trip + Miles_per_trip, apta, subset=train2)
summary(apta.tree3)
apta.tree4=rpart(Cost_per_trip ~ Population + Area + Fare_per_trip + Miles_per_trip, apta ,subset=train2)
fancyRpartPlot(apta.tree4)

![Initial Regression Tree](../../images/regression2.png)

This tree is quite a bit more exhaustive and has an interesting variation of nodes. Firstly, it is apparent that the length of a trip positively impacts that trip's cost, which could be expected. A trip's fare and the size of a city also have an impact on how much a city spends on each ride, which is also worth noting.

In [ ]:
cv.apta2<-cv.tree(apta.tree3)
plot(cv.apta2$size,cv.apta2$dev,type='b')
prune.apta2=prune.tree(apta.tree3,best=5)
plot(prune.apta2)
text(prune.apta2,pretty=0)
yhat2=predict(apta.tree3,newdata=apta[-train,])
apta.test2=apta[-train,"Cost_per_trip"]
plot(yhat2,apta.test2)
abline(0,1)
mean((yhat2-apta.test2)^2)

![Pruning via cv](../../images/regression2_prune.png)

![Pruned Tree](../../images/regression2_pruned.png)

After pruning, we can see that the number of nodes and leaves has been reduced quite significantly. The most notable change, however, is that `area` is no longer present in any of the conditions. This tells a different story than the tree above, eliminating the predictiveness of a city's size in determining the cost-effectiveness of its public transit system.

![Results of Pruned Tree](../../images/regression2_results.png)

Finally, from these results, it appears that this process did a bit better than the previous target variable. While there is still quite a bit of error to account for, there are certain leaves for which the predicted value seems to correlate with the actual value for cost per trip.

### Random Forests

Below is the code and output for random forests, starting with `Trips_per_capita` as the target variable:

In [ ]:
library(randomForest)
set.seed(1)
bag.apta=randomForest(Trips_per_capita ~ Population + Area + Fare_per_trip + Miles_per_trip, data=apta,subset=train, mtry=4, importance=TRUE)
bag.apta
yhat.bag = predict(bag.apta,newdata=apta[-train,])
plot(yhat.bag2, apta.test)
abline(0,1)
mean((yhat.bag-apta.test)^2)
bag.apta=randomForest(Trips_per_capita ~ Population + Area + Fare_per_trip + Miles_per_trip,data=apta,subset=train, mtry=4, ntree=25)
yhat.bag = predict(bag.apta,newdata=apta[-train,])
rf.apta=randomForest(Trips_per_capita ~ Population + Area + Fare_per_trip + Miles_per_trip, data=apta,subset=train,mtry=2,importance=TRUE)
yhat.rf = predict(rf.apta,newdata=apta[-train,])
mean((yhat.rf-apta.test)^2)
importance(rf.apta)
varImpPlot(rf.apta)

![Predicted Bagged Values](../../images/regression1_bag.png)

![Importance Plot](../../images/regression1_importance.png)

This importance plot tells us that `Population` and `Area` are the main predictors in determining `Trips_per_capita`. This result follows for both the proportion of MSE explained and the node purity. Thus, it is reasonable to infer that characteristics of a particular city are the primary determining factors of how much a public transit system is used.

In [ ]:
library(randomForest)
set.seed(1)
bag.apta2=randomForest(Cost_per_trip ~ Population + Area + Fare_per_trip + Miles_per_trip, data=apta,subset=train2, mtry=4, importance=TRUE)
bag.apta2
yhat.bag2 = predict(bag.apta2,newdata=apta[-train2,])
plot(yhat.bag2, apta.test2)
abline(0,1)
mean((yhat.bag2-apta.test2)^2)
bag.apta=randomForest(Cost_per_trip ~ Population + Area + Fare_per_trip + Miles_per_trip,data=apta,subset=train2, mtry=4, ntree=25)
yhat.bag = predict(bag.apta,newdata=apta[-train2,])
rf.apta=randomForest(Cost_per_trip ~ Population + Area + Fare_per_trip + Miles_per_trip, data=apta,subset=train2,mtry=2,importance=TRUE)
yhat.rf = predict(rf.apta,newdata=apta[-train2,])
mean((yhat.rf-apta.test2)^2)
importance(rf.apta)
varImpPlot(rf.apta)

![Predicted Bagged Values](../../images/regression2_bag.png)

![Importance Plot](../../images/regression2_importance.png)

This importance plot tells us that `Miles_per_trip` decisively accounts for the majority of Mean Squared Error and also has the most node purity. This follows intuition, as we could expect that the length of a trip plays a big role in how much that trip costs.

## Conclusions

When comparing these results to our data science question, it is important to think about what particular action items a city can take to improve their public transit system. Due to the presence of two target variables, we can analyze performance as a combination of two metrics.

Firstly, the proportional volume of public transit usage is a great indicator of how much a city's residence depend on those resources. From our analysis, it appears that the amount of ridership depends most heavily on the size of the city, with larger cities outperforming smaller cities in trips per capita.

As we will see in classification trees and through intuition, this is expected. Larger cities often have more robust public transit infrastructure, denser populations, and less reliance on travel by car. Unfortunately, it is then difficult to prescribe action items for a city, as success in this metric is largely dependent on the city's underlying characteristics.

Secondly, cost per trip tells us the financial viability of a public transit system. A city that offers more services per dollar is more likely to view that resource as necessary for its residents. Our results tell us that this is also largely a function of the city's characteristics. A trip is likely to cost more if it services longer trips.